### **Data Ingestion**

In [1]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import zipfile
import warnings

# import stabdard scaler
from sklearn.preprocessing import StandardScaler

#  import kmeans
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples


warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [10]:
# Download the data and give it a variable name
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
#file_path = "month_3\online_retail.csv"

In [11]:
online_retail = pd.read_excel(url)
online_retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [12]:
# making a copy of the original data
df = online_retail.copy()

### **2. Preliminary Data Analysis**

In [13]:
#df.to_csv("online_retail.csv", index=False)

In [14]:
# Checking the data
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [15]:
# Checking the shape of the data
df.shape

(541909, 8)

In [16]:
# Check for the datatype of each column
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [17]:
# Check for missing values
missing_values = df.isna().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})
missing_data.sort_values(by='Missing Values', ascending=False)

,Missing Values,Percentage
CustomerID,135080,24.926694
Description,1454,0.268311
StockCode,0,0.000000
InvoiceNo,0,0.000000
Quantity,0,0.000000
InvoiceDate,0,0.000000
UnitPrice,0,0.000000
Country,0,0.000000


In [18]:
# Keep only rows where InvoiceNo does not start with 'C' 
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')] 

In [19]:
df.InvoiceNo.unique()

array([536365, 536366, 536367, ..., 581585, 581586, 581587],
      shape=(22064,), dtype=object)

In [20]:
# Dropping the rows with missing values
df.dropna(inplace=True)

In [21]:
#Checking to see if the code above worked
missing_values = df.isna().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})
missing_data.sort_values(by='Missing Values', ascending=False)

,Missing Values,Percentage
InvoiceNo,0,0.0
StockCode,0,0.0
Description,0,0.0
Quantity,0,0.0
InvoiceDate,0,0.0
UnitPrice,0,0.0
CustomerID,0,0.0
Country,0,0.0


In [22]:
# Check for duplicate values or rows
df.duplicated().sum()

np.int64(5192)

In [23]:
# Remove the duplicates
df.drop_duplicates(inplace=True)

In [24]:
# Checking to see if the code above worked
df.duplicated().sum()

np.int64(0)

In [25]:
df.shape

(392732, 8)

### **3. Exploratory Data Analysis**

In [26]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

##### **a. Univariate analysis**

In [27]:
# lets' perform univariate analysis on the dataset starting with InvoiceNo
print(f"Unique Invoice Numbers: {df.InvoiceNo.nunique()}")
print(f"Total Number of Invoice Entries: {df.InvoiceNo.count()}")
print(f"Most Frequent Invoice Number: {df.InvoiceNo.mode()[0]} with {df.InvoiceNo.value_counts().max()} entries")

Unique Invoice Numbers: 18536
Total Number of Invoice Entries: 392732
Most Frequent Invoice Number: 576339 with 542 entries


In [28]:
# StockCode
print(f"Unique Stock Codes: {df.StockCode.nunique()}")
print(f"Total Number of Stock Code Entries: {df.StockCode.count()}")
print(f"Most Frequent Stock Code: {df.StockCode.mode()[0]} with {df.StockCode.value_counts().max()} entries")

Unique Stock Codes: 3665
Total Number of Stock Code Entries: 392732
Most Frequent Stock Code: 85123A with 2023 entries


In [29]:
# CustomerID
print(f"Unique Customer IDs: {df.CustomerID.nunique()}")
print(f"Total Number of Customer ID Entries: {df.CustomerID.count()}")
print(f"Most Frequent Customer ID: {df.CustomerID.mode()[0]} with {df.CustomerID.value_counts().max()} entries")

Unique Customer IDs: 4339
Total Number of Customer ID Entries: 392732
Most Frequent Customer ID: 17841.0 with 7676 entries


In [30]:
#Country
print(f"Unique Countries: {df.Country.nunique()}")
print(f"Total Number of Country Entries: {df.Country.count()}")
print(f"Most Frequent Country: {df.Country.mode()[0]} with {df.Country.value_counts().max()} entries")

Unique Countries: 37
Total Number of Country Entries: 392732
Most Frequent Country: United Kingdom with 349227 entries


In [31]:
# Total quantity sold for each product (top 10)
df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)

Description
PAPER CRAFT , LITTLE BIRDIE           80995
MEDIUM CERAMIC TOP STORAGE JAR        77916
WORLD WAR 2 GLIDERS ASSTD DESIGNS     54319
JUMBO BAG RED RETROSPOT               46078
WHITE HANGING HEART T-LIGHT HOLDER    36706
ASSORTED COLOUR BIRD ORNAMENT         35263
PACK OF 72 RETROSPOT CAKE CASES       33670
POPCORN HOLDER                        30919
RABBIT NIGHT LIGHT                    27153
MINI PAINT SET VINTAGE                26076
Name: Quantity, dtype: int64

In [32]:
# Let's add a new column 'TotalPrice' which is the product of Quantity and UnitPrice
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [33]:
# Let's perform bivariate analysis to see the relationship between Quantity and UnitPrice
product_analysis = df.groupby('Description').agg({
    'Quantity': 'sum',      # Total quantity sold for each product
    'InvoiceNo': 'count',   # Number of invoices for each product
    'UnitPrice': 'mean'     # Average unit price for each product
    }).round(2)

product_analysis.columns = ['Total_Quantity', 'Num_Transactions', 'Avg_Unit_Price']
product_analysis= product_analysis.sort_values('Total_Quantity', ascending=False)
product_analysis.head(10)

,Total_Quantity,Num_Transactions,Avg_Unit_Price
Description,,,
"PAPER CRAFT , LITTLE BIRDIE",80995,1,2.08
MEDIUM CERAMIC TOP STORAGE JAR,77916,198,1.22
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54319,472,0.29
JUMBO BAG RED RETROSPOT,46078,1615,2.02
WHITE HANGING HEART T-LIGHT HOLDER,36706,2016,2.89
ASSORTED COLOUR BIRD ORNAMENT,35263,1395,1.68
PACK OF 72 RETROSPOT CAKE CASES,33670,1050,0.55
POPCORN HOLDER,30919,653,0.84
RABBIT NIGHT LIGHT,27153,823,2.01


In [34]:
# Let's check for top 10 customers by total spending
top_customers = df.groupby('CustomerID')['TotalPrice'].sum().sort_values(ascending=False).head(10)
top_customers

CustomerID
14646.0    280206.02
18102.0    259657.30
17450.0    194390.79
16446.0    168472.50
14911.0    143711.17
12415.0    124914.53
14156.0    117210.08
17511.0     91062.38
16029.0     80850.84
12346.0     77183.60
Name: TotalPrice, dtype: float64

In [35]:
# Top items by total sales amount
top_items = df.groupby('Description')['TotalPrice'].sum().sort_values(ascending=False).head(10)
top_items

Description
PAPER CRAFT , LITTLE BIRDIE           168469.60
REGENCY CAKESTAND 3 TIER              142264.75
WHITE HANGING HEART T-LIGHT HOLDER    100392.10
JUMBO BAG RED RETROSPOT                85040.54
MEDIUM CERAMIC TOP STORAGE JAR         81416.73
POSTAGE                                77803.96
PARTY BUNTING                          68785.23
ASSORTED COLOUR BIRD ORNAMENT          56413.03
Manual                                 53419.93
RABBIT NIGHT LIGHT                     51251.24
Name: TotalPrice, dtype: float64

### **4. Preprocessing of our data for clustering**

**Clculation based on recency**

In [36]:
# let's check for the first and last transaction date

print(f"First transaction: {df['InvoiceDate'].min()}")
print(f"Last transaction: {df['InvoiceDate'].max()}")

# let's check the span in days(total number of transaction days)
print(f"Time span of transactions: {(df['InvoiceDate'].max() - df['InvoiceDate'].min())} days")

reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
print(f"Reference date (our 'today): {reference_date}")

First transaction: 2010-12-01 08:26:00
Last transaction: 2011-12-09 12:50:00
Time span of transactions: 373 days 04:24:00 days
Reference date (our 'today): 2011-12-10 12:50:00


In [37]:
# For each customer, find their most recent purchase date
recency = df.groupby('CustomerID')['InvoiceDate'].max().reset_index()
recency.columns = ['CustomerId', ' LastPurchaseDate']

# Calculate days since last purchase
recency['Recency'] = (reference_date - recency['LastPurchaseDate']).dt.days
recency.head(10)

KeyError: 'LastPurchaseDate'